Packages

In [1]:
import pandas as pd

Importing datasets

In [ ]:
agro = pd.read_csv('agro.csv')

In [11]:
agro = agro[agro['codmpio'].notnull()]
agro = agro.fillna(0)
agro

,codmpio,ano,ac_acelga,ac_achicoria,ac_achin,ac_achiote,ac_achira,ac_agraz,ac_aguacate,ac_aguacateh,...,vrf_mediano_productor_especial,vrf_microempresario,vrf_microempresario_mediano,vrf_microempresario_peq,vrf_mujer_rural,vrf_otros_productor_buenaventura,vrf_peq_productor,vrf_victimas_conflicto_gp,vrf_victimas_conflicto_mp,vrf_victimas_conflicto_pp
0,5001.0,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,0.0,103.970445,65.900,0.0,1953.625120,0.0,0.00,0.000000
1,5002.0,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,707.0,...,0.0,0.0,0.0,20.218740,93.910,0.0,5812.709665,0.0,130.00,215.000000
2,5004.0,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,148.0,...,0.0,0.0,0.0,0.000000,0.000,0.0,334.107291,0.0,70.00,24.250156
3,5021.0,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.500000,57.718,0.0,180.232707,0.0,31.25,70.074729
4,5030.0,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,...,0.0,0.0,0.0,30.600677,0.000,0.0,490.915946,0.0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21074,99773.0,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000,0.0,108.286465,0.0,0.00,0.000000
21075,99773.0,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.000000,0.000,0.0,406.927203,0.0,0.00,0.000000
21076,99773.0,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.000000,0.000,0.0,280.170092,0.0,0.00,6.660000
21077,99773.0,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000,0.0,539.417321,0.0,0.00,26.200000


In [12]:
agro.describe()

,codmpio,ano,ac_acelga,ac_achicoria,ac_achin,ac_achiote,ac_achira,ac_agraz,ac_aguacate,ac_aguacateh,...,vrf_mediano_productor_especial,vrf_microempresario,vrf_microempresario_mediano,vrf_microempresario_peq,vrf_mujer_rural,vrf_otros_productor_buenaventura,vrf_peq_productor,vrf_victimas_conflicto_gp,vrf_victimas_conflicto_mp,vrf_victimas_conflicto_pp
count,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,...,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000,21079.000000
mean,38091.937378,2012.009630,0.091170,0.001139,0.582551,0.219034,0.479047,0.155680,21.007085,2.678684,...,0.000772,3.628720,0.170965,29.347590,21.391760,0.154481,1242.085418,1.346277,4.661805,21.855462
std,26052.747832,5.476762,2.698371,0.089800,13.864426,5.672131,13.070575,5.871817,150.314643,57.047161,...,0.101928,28.427277,2.220386,161.610791,119.885995,14.725414,1697.119743,60.414530,41.067033,115.570872
min,5001.000000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15664.000000,2007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,184.878468,0.000000,0.000000,0.000000
50%,25799.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,676.690000,0.000000,0.000000,0.000000
75%,66045.000000,2017.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1622.976891,0.000000,0.000000,0.000000
max,99773.000000,2021.000000,300.000000,10.000000,1000.000000,350.000000,1000.000000,400.000000,6095.000000,4196.000000,...,14.716947,795.658383,177.372100,3811.205052,4715.741004,1921.356000,20835.903554,5000.000000,2197.178000,3618.847761
